# Predict of Google Stock Price using LSTM

In [24]:
import yfinance
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input, Dropout, SimpleRNN, LSTM, GRU
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

data = yfinance.download('GOOGL', start='2022-01-01', end='2024-12-30')
_close = data['Close'].values
data.head()

/tmp/ipykernel_10533/48049003.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yfinance.download('GOOGL', start='2022-01-01', end='2024-12-30')
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,GOOGL,GOOGL,GOOGL,GOOGL,GOOGL
Date,,,,,
2022-01-03,143.998322,144.851931,142.728576,144.061378,28646000
2022-01-04,143.410400,145.481610,142.732080,144.400065,28400000
2022-01-05,136.831268,143.509710,136.744874,143.430756,54618000
2022-01-06,136.803955,138.981442,135.623104,136.060093,37348000
2022-01-07,136.078445,137.500143,134.836508,137.199210,29760000


In [32]:
scaler = MinMaxScaler(feature_range=(0, 1))
close = scaler.fit_transform(_close.reshape(-1, 1))

def make_sequences(data, lookback_time_step=10, horizon_time_step=4):
    X, Y = [], []
    for i in range(len(data) - lookback_time_step - horizon_time_step):
        a = data[i:(i + lookback_time_step), 0]
        X.append(a)
        Y.append(data[i + lookback_time_step:i + lookback_time_step + horizon_time_step, 0])
    return np.array(X), np.array(Y)

x, y = make_sequences(close, lookback_time_step=60, horizon_time_step=4)

print(x.shape, y.shape)
print()
print(x[0])
print(y[0])

(687, 60) (687, 4)

[0.54028361 0.53508823 0.47694938 0.47670802 0.4702968  0.48479989
 0.49415952 0.50903099 0.48407593 0.49191727 0.46135382 0.45361725
 0.43774109 0.41179833 0.41576963 0.38181395 0.401982   0.39998097
 0.43812296 0.45525852 0.4757996  0.56668738 0.52359557 0.52537707
 0.48946453 0.49120208 0.51044856 0.4843653  0.44629802 0.45721127
 0.46671181 0.47662469 0.43099641 0.41225018 0.40707651 0.38754491
 0.43233051 0.44785137 0.45309556 0.44435835 0.44883422 0.44293646
 0.42544559 0.37692992 0.3833015  0.43872839 0.43003527 0.40757677
 0.373178   0.4016747  0.43750418 0.44240574 0.46247272 0.4622621
 0.4953182  0.48134192 0.51027314 0.5111593  0.50925065 0.51846573]
[0.51348947 0.48829291 0.49779736 0.52255555]


In [26]:
# ترکیب 80-20
xtrain, xvalidation, ytrain, yvalidation = train_test_split(x, y, test_size=0.2, random_state=42)
print(f"Training set size: {xtrain.shape[0]}")
print(f"Validation set size: {xvalidation.shape[0]}")

Training set size: 549
Validation set size: 138


In [27]:
# convert to keras shape [batch, timestep, features]
xtrain = np.reshape(xtrain, (xtrain.shape[0], xtrain.shape[1], 1))
xvalidation = np.reshape(xvalidation, (xvalidation.shape[0], xvalidation.shape[1], 1))

print(f"Training set shape after reshaping: {xtrain.shape}")
print(f"Validation set shape after reshaping: {xvalidation.shape}")

Training set shape after reshaping: (549, 60, 1)
Validation set shape after reshaping: (138, 60, 1)


In [28]:
public_callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5)
]

# Make SimpleRNN model

In [46]:
from keras_tuner import GridSearch, RandomSearch

In [51]:
def build_rnn_model(hp):
    simple_rnn_model = Sequential()
    simple_rnn_model.add(Input(shape=(xtrain.shape[1], 1)))
    for i in range(hp.Int("num_layers", 1, 3) - 1):
        simple_rnn_model.add(SimpleRNN(64, activation="relu", return_sequences=True))
    simple_rnn_model.add(SimpleRNN(64, activation="relu", return_sequences=False))
    # regression layer
    simple_rnn_model.add(Dense(4))
    simple_rnn_model.compile(
        optimizer=hp.Choice("optimizer", ["adam", "sgd"]), loss="mse", metrics=["mae"]
    )
    return simple_rnn_model

rnn_search = GridSearch(build_rnn_model, seed=42, objective="val_loss")
rnn_search.search(xtrain, ytrain, epochs=30, validation_data=(xvalidation, yvalidation))

Trial 6 Complete [00h 00m 12s]
val_loss: 0.0021016497630625963

Best val_loss So Far: 0.0012587010860443115
Total elapsed time: 00h 00m 59s


In [ ]:
def build_lstm_model(hp):
    lstm_model = Sequential()
    lstm_model.add(Input(shape=(xtrain.shape[1], 1)))
    for i in range(hp.Int("num_layers", 1, 3)-1):
        lstm_model.add(LSTM(hp.Choice("units", [32, 64]), activation="relu", return_sequences=True))
    lstm_model.add(LSTM(32, activation="relu", return_sequences=False))
    lstm_model.add(Dense(4))
    lstm_model.compile(optimizer=hp.Choice("optimizer", ["adam", "sgd"]), loss="mean_squared_error", metrics=["mae"])
    return lstm_model

lstm_search = GridSearch(build_lstm_model, seed=42, objective="val_loss")
lstm_search.search(xtrain, ytrain, epochs=30, validation_data=(xvalidation, yvalidation))

Reloading Tuner from ./untitled_project/tuner0.json


In [ ]:
def build_gru_model(hp):
    gru_model = Sequential()
    gru_model.add(Input(shape=(xtrain.shape[1], 1)))
    for i in range(hp.Int("num_layers", 1, 3)-1):
        gru_model.add(GRU(hp.Choice("units", [32, 64]), activation="relu", return_sequences=True))
    gru_model.add(GRU(32, activation="relu", return_sequences=False))
    gru_model.add(Dense(4))
    gru_model.compile(optimizer=hp.Choice("optimizer", ["adam", "sgd"]), loss="mean_squared_error", metrics=["mae"])
    return gru_model

gru_search = GridSearch(build_gru_model, seed=42, objective="val_loss")
gru_search.search(xtrain, ytrain, epochs=30, validation_data=(xvalidation, yvalidation))

Trial 10 Complete [00h 00m 24s]
val_loss: 0.04954695329070091

Best val_loss So Far: 0.0013030292466282845
Total elapsed time: 00h 03m 06s
